In [1]:
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
# Initialiser un DataFrame vide pour stocker les séries temporelles de chaque pixel
# Utiliser un MultiIndex pour les dates et les heures
all_dates = pd.date_range(start='2018-01-01', end='2018-01-20', freq='5T')
pixels = [(x, y) for x in range(300) for y in range(300)]
columns = pd.MultiIndex.from_tuples(pixels, names=['x', 'y'])

# Créer un DataFrame vide avec les pixels comme colonnes et les dates comme index
df_pixels = pd.DataFrame(index=all_dates, columns=columns)

# Fonction pour charger les données d'un jour spécifique et les ajouter au DataFrame global
def add_day_to_df(year, month, day, df_pixels):

    path = f"./datatest/"
    file = f'RR_IDF300x300_{year:04d}{month:02d}{day:02d}.npy'
    file_path = path + file
    try:
        RR = np.load(file_path) / 100.0

        RR[RR < 0] = np.nan  # Remplacer les valeurs non définies
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        return df_pixels  # Retourner le DataFrame inchangé si le fichier n'existe pas

    # Pour chaque pixel, ajouter ses valeurs à la série temporelle correspondante
    for x in range(300):
        for y in range(300):
            start_datetime = datetime(year, month, day)
            end_datetime = start_datetime + timedelta(days=1)
            time_range = pd.date_range(start=start_datetime, end=end_datetime, closed='left', freq='5T')

            # S'assurer que l'index temporel correspond au nombre de mesures
            if len(time_range) == RR.shape[0]:
                df_pixels.loc[time_range, (x, y)] = RR[:, x, y]

    return df_pixels

# Exemple d'ajout des données d'un jour spécifique
# df_pixels = add_day_to_df(2018, 12, 3, df_pixels)

# Pour traiter toute l'année, itérer sur chaque jour et ajouter les données
# Note : Cela peut prendre du temps en fonction de la taille des données et des performances de l'ordinateur

for month in range(1, 2):
    for day in range(1, 21):
        try:
            df_pixels = add_day_to_df(2018, month, day, df_pixels)
        except ValueError:  # Si la date n'est pas valide (ex : 30 février)
            continue

In [ ]:
def clustering(Dataset,k):
    # Initialiser et entraîner le modèle K-Means
    kmeans = KMeans(n_clusters=k, random_state=0)
    kmeans.fit(Dataset)

    # Obtenir les centres des clusters
    centers = kmeans.cluster_centers_
    print("Centres des clusters:")
    print(centers)

    # Obtenir les étiquettes des clusters pour chaque point
    labels = kmeans.labels_
    print("\nÉtiquettes des clusters:")
    print(labels)


In [ ]:
clustering(df_pixels,5)